In [ ]:
import pandas as pd
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import convert_to_unicode

# === Set your input BibTeX file path
bib_file_path = "refs.bib"

# === Parse BibTeX file
with open(bib_file_path, encoding='utf-8') as bibtex_file:
    parser = BibTexParser()
    parser.customization = convert_to_unicode
    bib_database = parser.parse_file(bibtex_file)

# === Convert to DataFrame
entries = bib_database.entries
df = pd.DataFrame(entries)
df = df.fillna('')
df.columns = [col.lower().capitalize() for col in df.columns]
df["Name"] = df["Title"]
# === Add custom annotation columns
df['Number'] = ''
df['No. of Citations'] = ''
df['Initial Letter'] = ''
# number of rows 
num_rows = len(df)
print(f"Number of entries in the BibTeX file: {num_rows}")
# print df columns
#print("Columns in the DataFrame:")	
#df.columns

# Sort rows by title (case-insensitive)
df = df.sort_values(by="Title", key=lambda col: col.str.lower())

# Fill 'Reference Number' with ascending numbers starting from 1
df["Number"] = range(1, len(df) + 1)

# Add initial letter column from title
df["Initial Letter"] = df["Title"].str.strip().str[0].str.upper()

df["Author"] = df["Author"].str.replace(r"\s+and\s+", ",", regex=True)
df["Author"] = df["Author"].apply(lambda x: ','.join(name.strip() for name in x.split(',')))


Number of entries in the BibTeX file: 79
Columns in the DataFrame:


In [2]:
# Count duplicate IDs
duplicate_ids = df[df.duplicated('Id', keep=False)]

if not duplicate_ids.empty:
    print("⚠️ Duplicate IDs found:")
    display(duplicate_ids)
else:
    print("✅ No duplicate IDs found.")

✅ No duplicate IDs found.


In [3]:
# === Save to CSV
df.to_csv("Notion.csv", index=False, encoding="utf-8-sig")
print("✅ Exported to 'Notion.csv'")

✅ Exported to 'Notion.csv'


In [4]:
from notion_client import Client

NOTION_TOKEN = "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"          
DATABASE_ID = "21195a58858c8060833ef73fd6b0b296"            

notion = Client(auth=NOTION_TOKEN)
existing = notion.databases.query(database_id=DATABASE_ID, page_size=1000)["results"]
id_to_page = {}

for page in existing:
    props = page["properties"]
    if "Id" in props and props["Id"]["rich_text"]:
        row_id = props["Id"]["rich_text"][0]["plain_text"]
        id_to_page[row_id] = page["id"]


In [5]:
def build_notion_properties(row):
    return {
        "Id": {"rich_text": [{"text": {"content": str(row["Id"])}}]},
        "Name": {"title": [{"text": {"content": str(row["Name"])}}]},
        "Abstract": {"rich_text": [{"text": {"content": str(row.get("Abstract", ""))}}]},
        "Doi": {"rich_text": [{"text": {"content": str(row.get("Doi", ""))}}]},
        "Url": {"url": str(row.get("Url", "")) or None},
        "Keywords": {"rich_text": [{"text": {"content": str(row.get("Keywords", ""))}}]},
        "Issn": {"rich_text": [{"text": {"content": str(row.get("Issn", ""))}}]},
        "Initial Letter": {"select": {"name": str(row.get("Initial Letter", ""))}} if row.get("Initial Letter") else None,
        "Author": {"multi_select": [
            {"name": name.strip()} for name in str(row.get("Author", "")).split(",") if name.strip()
        ]},
        "Year": {"number": int(row["Year"])} if pd.notnull(row["Year"]) else {"number": None},
        "Number": {"number": int(row["Number"])} if pd.notnull(row["Number"]) else {"number": None},
    }

# === Define format helpers ===
def format_rich_text(value):
    return [{"text": {"content": str(value)}}] if value else []

def format_multi_select(csv_tags):
    return [{"name": tag.strip()} for tag in str(csv_tags).split(",") if tag.strip()]

def safe_int(val):
    try:
        return int(val)
    except:
        return None
    
def format_rich_text(value, limit=2000):
    value = str(value)[:limit] if value else ""
    return [{"text": {"content": value}}]

# === Loop and sync ===
for _, row in df.iterrows():
    #print(f"Processing row: {row['Id']} - {row['Name']}")
    row_id = row["Id"]
    number_value = safe_int(row["Number"])
    name = str(row.get("Name", "")).strip() or "Untitled"
    url = str(row.get("Url", "")).strip()
    initial = str(row.get("Initial Letter", "")).strip()

    properties = {
        "Id": {"rich_text": format_rich_text(row["Id"])},
        "Name": {"title": format_rich_text(name)},
        "Abstract": {"rich_text": format_rich_text(row["Abstract"])},
        "Doi": {"rich_text": format_rich_text(row["Doi"])},
        "Issn": {"rich_text": format_rich_text(row["Issn"])},
        "Keywords": {"rich_text": format_rich_text(row["Keywords"])},
        "Author": {"multi_select": format_multi_select(row["Author"])},
        "Year": {"number": safe_int(row["Year"])},
        "Number": {"number": safe_int(row["Number"])},
        "Url": {"url": url if url else None},
    }

    if initial:
        properties["Initial Letter"] = {"select": {"name": initial}}

    if row_id in id_to_page:
        # ✅ Only update Number if entry exists
        notion.pages.update(
            page_id=id_to_page[row_id],
            properties={"Number": {"number": number_value}}
        )
        print(f"🔄 Updated Number for: {row_id}")
    else:
        notion.pages.create(parent={"database_id": DATABASE_ID}, properties=properties)
        print(f"➕ Created: {row_id}")

🔄 Updated Number for: ESC2020
🔄 Updated Number for: islam2021multislice
🔄 Updated Number for: Lundberg2017
🔄 Updated Number for: Sclafani2024
🔄 Updated Number for: Muscogiuri2022
🔄 Updated Number for: Cau2023-14
🔄 Updated Number for: Cau2024-15
🔄 Updated Number for: Laumer2022
🔄 Updated Number for: Cau2022-10
🔄 Updated Number for: Backhaus2019
🔄 Updated Number for: Cau2022-8
🔄 Updated Number for: Bbeiman1996
🔄 Updated Number for: Cau2024Base
🔄 Updated Number for: Hjagergaard2020
🔄 Updated Number for: Cau2024-3
🔄 Updated Number for: Cau2024-2
🔄 Updated Number for: Gopalakrishnan2015
🔄 Updated Number for: XIE2023102766
🔄 Updated Number for: Delong1988
🔄 Updated Number for: Cau2022-9
🔄 Updated Number for: Rawish2021
🔄 Updated Number for: Santoro2021
🔄 Updated Number for: zaman2024deep
🔄 Updated Number for: Diaz-Arocutipa2025
🔄 Updated Number for: Nussinovitch2011
🔄 Updated Number for: Kato2022
🔄 Updated Number for: Cau2025-12
🔄 Updated Number for: Cau2023-7
🔄 Updated Number for: Akashi201